## Imports

In [ ]:
import zipfile
import os
import sys
import random

import pandas as pd
import numpy as np


sys.path.append(".")

from datasetutil import gerar_estatiscas_df, substituir_valores_nulos , verify_gg4cc , save_zip2

## Metodos

In [ ]:
def read_data(ano,ds_files):
    zipf = 'consulta_cand_'+ano+'.zip' #2012
    zf = zipfile.ZipFile(ds_files+zipf) 
    first = True

    nl = zipfile.ZipFile.namelist(zf)
    for i in range(0,len(nl)):
        fn = nl[i]
        if ('txt' in fn) or ('csv' in fn):
            if not 'BRASIL' in fn:
#                 print(fn)
                z = pd.read_csv(zf.open(fn),header=0,encoding="iso-8859-1",sep=";",
                    error_bad_lines=False)
                if first:
                    first = False
                    df = z
                else:
                    df = pd.concat([df, z])
    return df

In [ ]:
def computar_divergencias(a,b,inter,debug=-1):

    atts = a.columns

    resumo = []
    
    if debug <= -1:
        l = len(inter)
    else:
        l = debug

    for cpf in list(inter)[0:l]:
        ra = a[a.NR_CPF_CANDIDATO == cpf]
        rb = b[b.NR_CPF_CANDIDATO == cpf]
        linha = {'cpf' : cpf}
        if debug > -1:
            print(ra.iloc[0][2] +'-'+ rb.iloc[0][2])
        for att in range(2,len(a.columns)):
            if (ra.iloc[0][att] != rb.iloc[0][att] ):
                linha[atts[att]] = 1
                if debug > -1:
                    print('\t['+atts[att]+'] ' + ra.iloc[0][att] +' -> '+  rb.iloc[0][att])
        resumo.append(linha)
        if debug > -1:
            print(' ')
    
    return pd.DataFrame(resumo).fillna(0)

## Leitura dos dados

In [ ]:
ds_files = 'D:\Dados\OneDrive\Doutorado\Datasets\\tse\\'

In [ ]:
# df08 = read_data('2008',ds_files)
# len(df08)
# df12 = read_data('2012',ds_files)
# len(df12)
# df = df08[df08[5] =='PB']
# df = df[df[7] =='CAMPINA GRANDE']

# # df = df12[df12[5] =='PB']
# # df = df[df[7] =='CAMPINA GRANDE']
# df.head(4).iloc[:,[21,20,22,23,5,7,10,14,13,25,36,30,32,34,36,37,39,27]]
# df.head(4).iloc[:,[21,5,7,10,14,13,25,36,30,32,34,36,37,39,27]]
# # nomes = ['UF','MUNICIPIO','ESTADO_CIVIL','NOME','APELIDO' ,'CPF1','PROFISSAO' ,'NASIMENTO','SEXO' ,'EDUCACAO','ESTADO_CIVIL' ,'NATURALIDADE','ESTADO' ,'CIDADE','CPF2' ,'','' ,'','' ,'',]
# # df.iloc[:, [5,7,10,14,13,25,36,30,32,34,36,37,39,27]]

In [ ]:
df14 = read_data('2014',ds_files)
df18 = read_data('2018',ds_files)

df14 = df14.iloc[:, [20,40,17,42,46,48,50,34,10,39,38,35,37,44,21]]
df18 = df18.iloc[:, [20,40,17,42,46,48,50,34,10,39,38,35,37,44,21]]

In [ ]:
inter = set(df14.iloc[:, 0 ]) & set(df18.iloc[:, 0 ])

In [ ]:
a = df14
b = df18

inter_a = df14[df14.NR_CPF_CANDIDATO.isin(list(inter))]
inter_b = df18[df18.NR_CPF_CANDIDATO.isin(list(inter))]

only_a = df14[~df14.NR_CPF_CANDIDATO.isin(list(inter))]
only_b = df18[~df18.NR_CPF_CANDIDATO.isin(list(inter))]

In [ ]:
print('a =',len(a))
print('b =',len(b))
print('a ∩ b =',len(inter))
print(len(inter_a),len(inter_b)) # dois cpf

In [ ]:
ds_files = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep+'workspace'+os.sep+'tl@pprl'+os.sep+'datasets'+os.sep
ds_files += 'tse' + os.sep

porcentagem_match = len(inter)/ (len(a) + len(b))
porcentagem_unmatch = 1 - (porcentagem_match)


for n in [2000,5000,10000,25000]:
    # n = 2000
    m = int(n * porcentagem_match)
    um = int(n * porcentagem_unmatch)

    int(um/2)

    random_inter = random.sample(inter,m)
    dups_a = a[a.NR_CPF_CANDIDATO.isin(list(random_inter))]
    dups_b = b[b.NR_CPF_CANDIDATO.isin(list(random_inter))]


    sdfa = pd.concat([only_a.sample(int(um/2)),
                only_b.sample(int(um/2)),
                dups_a
                ])

    sdfb = pd.concat([only_a.sample(int(um/2)),
                only_b.sample(int(um/2)),
                dups_b
                ])

    # gerando gabarito
    linhas = []
    for id_ in dups_a.NR_CPF_CANDIDATO.unique():
        linha = {'id1':id_,'id2':id_}
        linhas.append(linha)

    gabarito = pd.DataFrame(linhas)

    #gerando estatisticas
    nan_list = sdfa.isnull().sum()
    estatiscas = sdfa.describe(include='object')
    est_a = gerar_estatiscas_df(nan_list,estatiscas)

    nan_list = sdfb.isnull().sum()
    estatiscas = sdfb.describe(include='object')
    est_b = gerar_estatiscas_df(nan_list,estatiscas)

    #filtrando dados
    sdfa = sdfa.iloc[:,[0,2,3,4,7,11,12]]
    sdfb = sdfb.iloc[:,[0,2,3,4,7,11,12]]

    of = ds_files + 'processed_tse-'+str(n)+'-8.zip'
    save_zip2(sdfa,sdfb,gabarito,est_a,est_b,of)

estatisticas

dados

In [ ]:
z = computar_divergencias(sdfa,sdfb,random_inter)
z.to_csv(ds_files+'sujeira_processed_tse-10k-8.csv', index=False)

In [ ]:
z.sum()

# Estatisticas

In [ ]:
a = df14[df14.NR_CPF_CANDIDATO.isin(list(inter))]
b = df18[df18.NR_CPF_CANDIDATO.isin(list(inter))]

del a['NR_IDADE_DATA_POSSE']
del b['NR_IDADE_DATA_POSSE']
del a['NM_EMAIL']
del b['NM_EMAIL']

In [ ]:
print('a =',len(a))
print('b =',len(b))
print('a ∩ b =',len(inter))

In [ ]:
print('a =',len(df14))
print('b =',len(df18))
print('a ∩ b =',len(a))

In [ ]:
resumo = computar_divergencias(a,b,inter)

### Estatiscas da Intercessão

In [ ]:
resumo.drop('cpf',1).sum(), len(resumo)

In [ ]:
z = computar_divergencias(a,b,inter,debug=10)